# ETL (TRANSFORMATION) PROCESS
In this stage, we will transform the data to fit our analysis needs. This includes cleaning, enrichment, Structuring, filtering and Categorization both the raw data and the incremental data.

For this process, we will begin with the `raw data`, applying various transformations to prepare it for analysis. The transformation process will include the following steps:

**1) Data Cleaning:**
   - Remove duplicates.
   - Handling missing values by either filling them with a default value or removing the rows/columns with missing data.

In [1]:
# import necessary libraries
import pandas as pd
import numpy as np

# load the dataset
raw_data = pd.read_csv("Data/raw_data.csv")

# check for missing values
missing_values = raw_data.isnull().sum()
# print the missing values
print(f"Missing values in each column:\n{missing_values}")

# check for duplicate rows
duplicate_rows = raw_data.duplicated().sum()
# print the number of duplicate rows
print(f"Number of duplicate rows: {duplicate_rows}")

Missing values in each column:
order_id          0
customer_name     1
product           0
quantity         26
unit_price       35
order_date        1
region           25
dtype: int64
Number of duplicate rows: 1


To handle missing values, we will do the following:
- We will fill the missing customer names with "Unknown".
- We will fill the missing value for quantity and price with the median since there is possibility of outliers.
- We will forward fill the missing values for the date column to ensure continuity in the time series data.
- We will fill region with "Unknown" if it is missing to avoid bias in analysis.

To remove duplicates, we will drop any rows that have the same values across all columns.

In [2]:
# Fill missing customer_name with 'unknown'
raw_data['customer_name'] = raw_data['customer_name'].fillna('unknown')

# Fill quantity with the median of the column
raw_data['quantity'] = raw_data['quantity'].fillna(raw_data['quantity'].median())

# Fill unit_price with the median of the column
raw_data['unit_price'] = raw_data['unit_price'].fillna(raw_data['unit_price'].median())

# Forward fill the missing values in the date column
raw_data['order_date'] = raw_data['order_date'].ffill()

# Fill region with 'Unknown'
raw_data['region'] = raw_data['region'].fillna('Unknown')

# Drop duplicate rows
raw_data_cleaned = raw_data.drop_duplicates()

# Display the first few rows of the cleaned dataset
raw_data_cleaned.head()

,order_id,customer_name,product,quantity,unit_price,order_date,region
0,1,Diana,Tablet,2.0,500.0,2024-01-20,South
1,2,Eve,Laptop,2.0,500.0,2024-04-29,North
2,3,Charlie,Laptop,2.0,250.0,2024-01-08,Unknown
3,4,Eve,Laptop,2.0,750.0,2024-01-07,West
4,5,Eve,Tablet,3.0,500.0,2024-03-07,South


**2) Data Enrichment:**

This involves adding new columns or modify existing ones to enhance the dataset. In this case, we will add a new column for the total price.

    `total_price = quantity * price`


In [3]:
# Calculate total price as quantity multiplied by unit price
raw_data_cleaned.loc[:, 'total_spend'] = raw_data_cleaned['quantity'] * raw_data_cleaned['unit_price']
raw_data_cleaned.head()

C:\Users\admin\AppData\Local\Temp\ipykernel_17680\1392200383.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data_cleaned.loc[:, 'total_spend'] = raw_data_cleaned['quantity'] * raw_data_cleaned['unit_price']


,order_id,customer_name,product,quantity,unit_price,order_date,region,total_spend
0,1,Diana,Tablet,2.0,500.0,2024-01-20,South,1000.0
1,2,Eve,Laptop,2.0,500.0,2024-04-29,North,1000.0
2,3,Charlie,Laptop,2.0,250.0,2024-01-08,Unknown,500.0
3,4,Eve,Laptop,2.0,750.0,2024-01-07,West,1500.0
4,5,Eve,Tablet,3.0,500.0,2024-03-07,South,1500.0


**3) Structuring Data:**

This step involves organizing the data into a structured format that is easy to analyze. We will ensure that the data types of each column are appropriate for analysis by converting the date column to a datetime format, and ensuring that numerical columns are of the correct type (e.g., integers or floats).


In [4]:
# converting 'order_date' column to datetime format 
raw_data_cleaned.loc[:, 'order_date'] = pd.to_datetime(raw_data_cleaned['order_date'], errors='coerce')
raw_data_cleaned.head()

,order_id,customer_name,product,quantity,unit_price,order_date,region,total_spend
0,1,Diana,Tablet,2.0,500.0,2024-01-20 00:00:00,South,1000.0
1,2,Eve,Laptop,2.0,500.0,2024-04-29 00:00:00,North,1000.0
2,3,Charlie,Laptop,2.0,250.0,2024-01-08 00:00:00,Unknown,500.0
3,4,Eve,Laptop,2.0,750.0,2024-01-07 00:00:00,West,1500.0
4,5,Eve,Tablet,3.0,500.0,2024-03-07 00:00:00,South,1500.0


**4) Categorization:**

This step invloves categorizing the data into meaningful groups. We will categorize the data based on the region and product type to facilitate analysis. In this case, we will create customer tiers based on the total price spent by each customer. The tiers will be defined as follows:
- **Bronze**: Total price < 100
- **Silver**: 100 <= Total price < 500
- **Gold**: Total price >= 500
- **Platinum**: Total price >= 1000


In [5]:
# Create Customer Tier Based on Spend
def assign_tier(spend):
    if spend <= 100:
        return 'Bronze'
    elif spend <= 500:
        return 'Silver'
    elif spend <= 1000:
        return 'Gold'
    else:
        return 'Platinum'

# Apply the function to create a new column 'customer_tier'
raw_data_cleaned.loc[:, 'customer_tier'] = raw_data_cleaned['total_spend'].apply(assign_tier)

# Save the cleaned data to a new CSV file
raw_data_cleaned.to_csv("Transformed/transformed_full.csv", index=False)
raw_data_cleaned.head()

C:\Users\admin\AppData\Local\Temp\ipykernel_17680\4179876910.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  raw_data_cleaned.loc[:, 'customer_tier'] = raw_data_cleaned['total_spend'].apply(assign_tier)


,order_id,customer_name,product,quantity,unit_price,order_date,region,total_spend,customer_tier
0,1,Diana,Tablet,2.0,500.0,2024-01-20 00:00:00,South,1000.0,Gold
1,2,Eve,Laptop,2.0,500.0,2024-04-29 00:00:00,North,1000.0,Gold
2,3,Charlie,Laptop,2.0,250.0,2024-01-08 00:00:00,Unknown,500.0,Silver
3,4,Eve,Laptop,2.0,750.0,2024-01-07 00:00:00,West,1500.0,Platinum
4,5,Eve,Tablet,3.0,500.0,2024-03-07 00:00:00,South,1500.0,Platinum


In the next section, we will implement the transformation on the `incremental data` to ensure that the new data is consistent with the existing dataset. This will involve applying the same cleaning, enrichment, structuring, and categorization steps to the incremental data.

**1) Data Cleaning:**
   - Handle missing values in the same way as the raw data.
   - Since there were no duplicates in the incremental data, we will not remove any duplicates.


In [6]:
# loading incremental data
incremental_data = pd.read_csv("Data/incremental_data.csv")

# Check for missing values in incremental data
missing_values_incremental = incremental_data.isnull().sum()
# Print the missing values in incremental data
print(f"Missing values in incremental data:\n{missing_values_incremental}")

# Check for duplicate rows in incremental data
duplicate_rows_incremental = incremental_data.duplicated().sum()
# Print the number of duplicate rows in incremental data
print(f"Number of duplicate rows in incremental data: {duplicate_rows_incremental}")

# Fill missing customer_name with 'unknown' in incremental data
incremental_data['customer_name'] = incremental_data['customer_name'].fillna('Unknown')

# Fill quantity with the median of the column in incremental data
incremental_data['quantity'] = incremental_data['quantity'].fillna(incremental_data['quantity'].median())

# Fill region with 'Unknown' in incremental data
incremental_data['region'] = incremental_data['region'].fillna('Unknown')
incremental_data.head()

Missing values in incremental data:
order_id         0
customer_name    6
product          0
quantity         4
unit_price       0
order_date       0
region           2
dtype: int64
Number of duplicate rows in incremental data: 0


,order_id,customer_name,product,quantity,unit_price,order_date,region
0,101,Alice,Laptop,1.5,900.0,2024-05-09,Central
1,102,Unknown,Laptop,1.0,300.0,2024-05-07,Central
2,103,Unknown,Laptop,1.0,600.0,2024-05-04,Central
3,104,Unknown,Tablet,1.5,300.0,2024-05-26,Central
4,105,Heidi,Tablet,2.0,600.0,2024-05-21,North


**2) Structuring Data:**
   - Ensure that the data types of each column are appropriate for analysis, similar to the raw data.
   

In [7]:
# Convert 'order_date' column to datetime format in incremental data
incremental_data.loc[:, 'order_date'] = pd.to_datetime(incremental_data['order_date'], errors='coerce')
incremental_data.head()

,order_id,customer_name,product,quantity,unit_price,order_date,region
0,101,Alice,Laptop,1.5,900.0,2024-05-09 00:00:00,Central
1,102,Unknown,Laptop,1.0,300.0,2024-05-07 00:00:00,Central
2,103,Unknown,Laptop,1.0,600.0,2024-05-04 00:00:00,Central
3,104,Unknown,Tablet,1.5,300.0,2024-05-26 00:00:00,Central
4,105,Heidi,Tablet,2.0,600.0,2024-05-21 00:00:00,North


**3) Filtering:**
   - Filter the incremental data to include only the relevant columns for analysis. 
   - In this case, we will drop the `customer_name` column assuming I am doing anonymous analysis and do not need to retain customer names. I will also drop records where the `region` is "Unknown" to ensure that the analysis is focused on known regions.

In [8]:
# Drop customer_name column from incremental data
incremental_data = incremental_data.drop(columns=['customer_name'])

# Drop records where the region is 'Unknown' 
incremental_data = incremental_data[incremental_data['region'] != 'Unknown']
incremental_data.head()


,order_id,product,quantity,unit_price,order_date,region
0,101,Laptop,1.5,900.0,2024-05-09 00:00:00,Central
1,102,Laptop,1.0,300.0,2024-05-07 00:00:00,Central
2,103,Laptop,1.0,600.0,2024-05-04 00:00:00,Central
3,104,Tablet,1.5,300.0,2024-05-26 00:00:00,Central
4,105,Tablet,2.0,600.0,2024-05-21 00:00:00,North


**3) Categorization:**
   - For the Incremental data, we will use One-Hot Encoding to categorize the data based on the region and prduct type. This will create binary columns for each category, allowing for easier analysis.

In [9]:
# Convert categorical columns to dummy variables 
incremental_data = pd.get_dummies(incremental_data, columns=['product', 'region'], prefix=['prod', 'reg'])
incremental_data.head()

,order_id,quantity,unit_price,order_date,prod_Laptop,prod_Tablet,reg_Central,reg_North
0,101,1.5,900.0,2024-05-09 00:00:00,True,False,True,False
1,102,1.0,300.0,2024-05-07 00:00:00,True,False,True,False
2,103,1.0,600.0,2024-05-04 00:00:00,True,False,True,False
3,104,1.5,300.0,2024-05-26 00:00:00,False,True,True,False
4,105,2.0,600.0,2024-05-21 00:00:00,False,True,False,True


**4) Data Enrichment:**
   - Add the `total_price` column to the incremental data in the same way as the raw data.
   - By doing this, we can calculate the total spend per region and product type, which will be useful for analysis.

In [10]:
# Adding total_spend column to incremental data
incremental_data.loc[:, 'total_spend'] = incremental_data['quantity'] * incremental_data['unit_price']

# Saving the transformed incremental data to a new CSV file
incremental_data.to_csv("Transformed/transformed_incremental.csv", index=False)
incremental_data.head()

,order_id,quantity,unit_price,order_date,prod_Laptop,prod_Tablet,reg_Central,reg_North,total_spend
0,101,1.5,900.0,2024-05-09 00:00:00,True,False,True,False,1350.0
1,102,1.0,300.0,2024-05-07 00:00:00,True,False,True,False,300.0
2,103,1.0,600.0,2024-05-04 00:00:00,True,False,True,False,600.0
3,104,1.5,300.0,2024-05-26 00:00:00,False,True,True,False,450.0
4,105,2.0,600.0,2024-05-21 00:00:00,False,True,False,True,1200.0
